In [1]:
reset -fs

In [140]:
# import libraries
import requests
import pandas as pd
import numpy as np
import time
import sqlite3
from bs4 import BeautifulSoup
import os

In [142]:
os.getcwd()

'/Users/chloebergsma-safar/Desktop/Metis/NBM_Unsupervised_Project/NLP-Unsupervised-Learning'

In [118]:
# read in articles_df
articles_df = pd.read_csv('modern_love_articles.csv')

In [119]:
articles_df.shape

(1170, 11)

In [120]:
articles_df.article_lead_paragraph.str.len().describe()

count    1159.000000
mean     288.638481 
std      151.287701 
min      1.000000   
25%      180.000000 
50%      256.000000 
75%      379.000000 
max      877.000000 
Name: article_lead_paragraph, dtype: float64

In [121]:
pd.set_option("display.max_colwidth", 1)

In [122]:
# check how many articles reference the podcast
articles_df['article_url'].str.contains('podcast').sum()

118

In [123]:
# remove articles that reference the podcast
articles_df = articles_df[~articles_df.article_url.str.contains('podcast')]

In [124]:
articles_df.shape

(1052, 11)

### Further analyze rows to ensure they're all Modern Love articles

In [125]:
# see how many articles have 'Interactive' as the type material, meaning it's not an actual Modern Love article
len(articles_df[articles_df.article_type_material.str.contains('Interactive')])

6

In [126]:
# remove articles that have 'Interactive' as the type material
articles_df = articles_df[~articles_df.article_type_material.str.contains('Interactive')]

In [127]:
# see how many articles have 'contest' in the URL, since we'll want to remove those
len(articles_df[articles_df.article_url.str.contains('contest')])

3

In [128]:
# remove articles that have 'contest' in the url
articles_df = articles_df[~articles_df.article_url.str.contains('contest')]

In [129]:
# remove article whose article_abstract is 'Try the 36 Questions on the Way to Love'
articles_df = articles_df[~articles_df.article_headline.str.contains('Try the 36')]

In [130]:
# remove article whose article_headline is 'College Students: Send Us Your 100-Word Love Stories'
articles_df = articles_df[~articles_df.article_headline.str.contains('College Students: Send Us Your 100-Word Love Stories')]

In [131]:
articles_df.shape

(1040, 11)

In [132]:
# see how many articles have 'Tiny Love Stories' or '100 words' in article_abstract
len(articles_df[(articles_df.article_abstract.str.contains('Tiny Love Stories')) | 
               (articles_df.article_abstract.str.contains('100 words'))])

171

In [133]:
# save df for tiny love stories
tiny_love = articles_df[(articles_df.article_abstract.str.contains('Tiny Love Stories')) | 
               (articles_df.article_abstract.str.contains('100 words'))]

In [134]:
tiny_love.shape

(171, 11)

In [135]:
# save df for non-tiny love stories
regular_love = articles_df[~((articles_df.article_abstract.str.contains('Tiny Love Stories')) | 
               (articles_df.article_abstract.str.contains('100 words')))]

In [136]:
regular_love.shape

(869, 11)

In [1]:
regular_love.sample(5)

NameError: name 'regular_love' is not defined

# Extract full paragraphs from regular Modern Love essays

In [23]:
regular_love_urls = list(regular_love['article_url'])

In [24]:
regular_love_urls[:10]

['https://www.nytimes.com/2022/02/25/style/modern-love-seeking-a-father-for-my-child-relationship-optional.html',
 'https://www.nytimes.com/2022/02/18/style/modern-love-i-tried-so-hard-to-be-good.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-gay-couple.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-confidence-self-love.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-black-boyfriend-applications.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-queer-dating-open-relationship.html',
 'https://www.nytimes.com/2022/02/11/style/what-is-black-love-today.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-bond-father.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-black-boys.html',
 'https://www.nytimes.com/2022/02/11/style/modern-love-healer-heartbreak.html']

In [27]:
# loop through articles to extract paragraphs and save them in a dictionary with their corresponding url

regular_love_paragraphs = {}
text_list = []

for reg_url in regular_love_urls[:100]:
    response_reg = requests.get(reg_url)
    time.sleep(4)
    soup_reg = BeautifulSoup(response_reg.text, "lxml")
    paragraphs_reg = [par for par in soup_reg.find_all('p',class_="css-axufdj evys1bk0")]
    text = ' '.join([elem.text for elem in paragraphs_reg])
    text_list.append(text)

for a, b in zip(regular_love_urls, text_list):
    regular_love_paragraphs[a] = [b]

In [30]:
# check first five key, value pairs in regular_love_paragraphs dict to make sure it's correct

{k: regular_love_paragraphs[k] for k in list(regular_love_paragraphs)[:5]}

{'https://www.nytimes.com/2022/02/25/style/modern-love-seeking-a-father-for-my-child-relationship-optional.html': ['Two days before I left South Dakota, Rex and I sat talking beneath the open hatchback of my car. In the distance, a lightning storm moved toward us over the open expanse of the Great Plains, churning the sky a murky purple. He was talking passionately about lithium batteries. The more he talked, the less he and I seemed to have in common. I considered myself someone who could become interested in almost anything, especially when I was attracted to the person speaking. But now I asked myself: Did I care about batteries? He and I were volunteers on the Pine Ridge Reservation, building and repairing infrastructure. He had been the first person to greet me when I arrived at the end of the long, dirt road. When he climbed out of the cab of a skid loader, and I saw his face, my body warmed. In the lyrical version of what happened as the storm approached, we would have stopped t

In [31]:
# loop through articles to extract paragraphs and save them in a dictionary with their corresponding url

regular_love_paragraphs_2 = {}
text_list_2 = []

for reg_url in regular_love_urls[100:400]:
    response_reg = requests.get(reg_url)
    time.sleep(4)
    soup_reg = BeautifulSoup(response_reg.text, "lxml")
    paragraphs_reg = [par for par in soup_reg.find_all('p',class_="css-axufdj evys1bk0")]
    text = ' '.join([elem.text for elem in paragraphs_reg])
    text_list_2.append(text)

for a, b in zip(regular_love_urls[100:400], text_list_2):
    regular_love_paragraphs_2[a] = [b]

In [32]:
len(regular_love_paragraphs_2)

300

In [37]:
# create a dataframe out of the dictionary containing each article and its url in regular_love_paragraphs

regular_love_df = pd.DataFrame(regular_love_paragraphs).T  #transpose
regular_love_df = regular_love_df.reset_index()
regular_love_df.columns = ['article_url','article']
regular_love_df.head()

article_url  \
0  https://www.nytimes.com/2022/02/25/style/modern-love-seeking-a-father-for-my-child-relationship-optional.html   
1  https://www.nytimes.com/2022/02/18/style/modern-love-i-tried-so-hard-to-be-good.html                            
2  https://www.nytimes.com/2022/02/11/style/modern-love-gay-couple.html                                            
3  https://www.nytimes.com/2022/02/11/style/modern-love-confidence-self-love.html                                  
4  https://www.nytimes.com/2022/02/11/style/modern-love-black-boyfriend-applications.html                          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [56]:
# create a dataframe out of the dictionary containing each article and its url in regular_love_paragraphs_2

regular_love_df_2 = pd.DataFrame(regular_love_paragraphs_2).T  #transpose
regular_love_df_2 = regular_love_df_2.reset_index()
regular_love_df_2.columns = ['article_url','article']

In [40]:
# append regular_love_df_2 to regular_love_df

regular_love_df_merged = regular_love_df.append(regular_love_df_2)

In [45]:
regular_love_df_merged.shape

(400, 2)

In [57]:
regular_love_df_merged.sample(3)

article_url  \
208  https://www.nytimes.com/2016/07/31/fashion/modern-love-words-with-friends.html            
117  https://www.nytimes.com/2018/01/05/style/modern-love-how-30-blocks-became-30-years.html   
269  https://www.nytimes.com/2015/05/31/style/after-a-first-time-many-second-thoughts.html     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [47]:
# loop through articles to extract paragraphs and save them in a dictionary with their corresponding url

regular_love_paragraphs_3 = {}
text_list_3 = []

for reg_url in regular_love_urls[400:700]:
    response_reg = requests.get(reg_url)
    time.sleep(4)
    soup_reg = BeautifulSoup(response_reg.text, "lxml")
    paragraphs_reg = [par for par in soup_reg.find_all('p',class_="css-axufdj evys1bk0")]
    text = ' '.join([elem.text for elem in paragraphs_reg])
    text_list_3.append(text)

for a, b in zip(regular_love_urls[400:700], text_list_3):
    regular_love_paragraphs_3[a] = [b]

In [58]:
# create a dataframe out of the dictionary containing each article and its url in regular_love_paragraphs_3

regular_love_df_3 = pd.DataFrame(regular_love_paragraphs_3).T  #transpose
regular_love_df_3 = regular_love_df_3.reset_index()
regular_love_df_3.columns = ['article_url','article']

In [59]:
# append regular_love_df_3 to regular_love_df_merged

regular_love_df_merged = regular_love_df_merged.append(regular_love_df_3)

In [62]:
regular_love_df_merged.sample(3)

article_url  \
204  https://www.nytimes.com/2009/05/31/fashion/31lovesub.html                    
201  https://www.nytimes.com/2009/06/21/fashion/21love.html                       
24   https://www.nytimes.com/2012/11/18/fashion/a-role-he-was-born-to-play.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [63]:
# loop through articles to extract paragraphs and save them in a dictionary with their corresponding url

regular_love_paragraphs_4 = {}
text_list_4 = []

for reg_url in regular_love_urls[700:]:
    response_reg = requests.get(reg_url)
    time.sleep(4)
    soup_reg = BeautifulSoup(response_reg.text, "lxml")
    paragraphs_reg = [par for par in soup_reg.find_all('p',class_="css-axufdj evys1bk0")]
    text = ' '.join([elem.text for elem in paragraphs_reg])
    text_list_4.append(text)

for a, b in zip(regular_love_urls[700:], text_list_4):
    regular_love_paragraphs_4[a] = [b]

In [64]:
# create a dataframe out of the dictionary containing each article and its url in regular_love_paragraphs_4

regular_love_df_4 = pd.DataFrame(regular_love_paragraphs_4).T  #transpose
regular_love_df_4 = regular_love_df_4.reset_index()
regular_love_df_4.columns = ['article_url','article']

In [70]:
regular_love_df_4.sample(3)

article_url  \
100  https://www.nytimes.com/2006/03/05/fashion/sundaystyles/a-girl-could-get-cornered-in-a-tiny-house.html   
59   https://www.nytimes.com/2006/09/03/fashion/03love.html                                                   
87   https://www.nytimes.com/2006/05/21/fashion/sundaystyles/21love.html                                      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [117]:
# append regular_love_df_4 to regular_love_df_merged

regular_love_df_merged = regular_love_df_merged.append(regular_love_df_4)

NameError: name 'regular_love_df_merged' is not defined

In [71]:
regular_love_df_merged.shape

(867, 2)

In [72]:
# save merged df as csv
regular_love_df_merged.to_csv('regular_love_df.csv',index=False)

# Extract Tiny Love Stories

In [22]:
tiny_love_urls = list(tiny_love['article_url'])

In [23]:
tiny_love_urls[:10]

['https://www.nytimes.com/2022/02/22/style/tiny-modern-love-stories-he-was-my-mothers-type-not-mine.html',
 'https://www.nytimes.com/2022/02/15/style/tiny-modern-love-stories-never-good-enough-never-skinny-enough.html',
 'https://www.nytimes.com/2022/02/08/style/tiny-modern-love-stories-the-third-time-she-left-me.html',
 'https://www.nytimes.com/2022/02/01/style/tiny-modern-love-stories-i-cried-without-knowing-why.html',
 'https://www.nytimes.com/2022/01/25/style/tiny-modern-love-stories-an-invitation-to-his-bed.html',
 'https://www.nytimes.com/2022/01/18/style/tiny-modern-love-stories-be-grateful-we-have-different-last-names.html',
 'https://www.nytimes.com/2022/01/11/style/tiny-modern-love-stories-long-after-my-ex-had-stopped-caring.html',
 'https://www.nytimes.com/2022/01/04/style/tiny-modern-love-stories-i-chased-her-shamelessly.html',
 'https://www.nytimes.com/2021/12/31/style/tiny-love-stories-fresh-starts.html',
 'https://www.nytimes.com/2021/12/21/style/tiny-modern-love-stories

In [97]:
# loop through articles to extract stories and save them in lists

tiny_urls = []
tiny_stories = []

for tiny_url in tiny_love_urls:
    response_tiny = requests.get(tiny_url)
    time.sleep(4)
    soup_tiny = BeautifulSoup(response_tiny.text, "lxml")
    paragraphs_tiny = [par for par in soup_tiny.find_all('p',class_="css-axufdj evys1bk0")]
    text = [elem.text for elem in paragraphs_tiny]
    tiny_stories.append(text)
    num_url_copies = len(text)
    urls = [tiny_url] * num_url_copies
    tiny_urls.append(urls)

In [101]:
# convert tiny_urls and tiny_stories lists to a dataframe

xx = [item for sublist in tiny_urls for item in sublist]
yy = [item for sublist in tiny_stories for item in sublist]
zz = {'article_url':xx,'article':yy}
tiny_love_df = pd.DataFrame(zz)

In [102]:
# save df as csv
tiny_love_df.to_csv('tiny_love_df.csv',index=False)

# Merge regular_love and regular_love_df_merged dfs

In [113]:
regular_love.columns

Index(['article_id', 'article_url', 'article_headline', 'article_abstract',
       'article_snippet', 'article_lead_paragraph', 'article_pub_date',
       'article_news_desk', 'article_section', 'article_type_material',
       'article_word_count'],
      dtype='object')

In [143]:
# read in regular_love_df_merged
regular_love_df_merged = pd.read_csv('regular_love_df.csv')

In [144]:
regular_love_df_merged.columns

Index(['article_url', 'article'], dtype='object')

In [145]:
regular_love_df_fin = regular_love_df_merged.merge(regular_love,how='left',on='article_url')

In [147]:
# save df as csv
regular_love_df_fin.to_csv('regular_love_df_final.csv',index=False)

# Merge tiny_love and tiny_love_dfs

In [103]:
tiny_love.columns

Index(['article_id', 'article_url', 'article_headline', 'article_abstract',
       'article_snippet', 'article_lead_paragraph', 'article_pub_date',
       'article_news_desk', 'article_section', 'article_type_material',
       'article_word_count'],
      dtype='object')

In [104]:
tiny_love_df.columns

Index(['article_url', 'article'], dtype='object')

In [108]:
tiny_love_df_fin = tiny_love_df.merge(tiny_love,how='left',on='article_url')

In [112]:
# save df as csv
tiny_love_df_fin.to_csv('tiny_love_df_final.csv',index=False)

# Merge regular_love_df_fin and tiny_love_df_fin dfs

In [152]:
len(tiny_love_df_fin.columns)

12

In [153]:
len(regular_love_df_fin.columns)

12

In [154]:
len(set(regular_love_df_fin.columns).intersection(set(tiny_love_df_fin.columns)))

12

In [155]:
modern_love_df = regular_love_df_fin.append(tiny_love_df_fin)

In [156]:
modern_love_df.shape

(1615, 12)

In [160]:
# save df as csv
modern_love_df.to_csv('modern_love_df.csv',index=False)